# Project AI

Import required sources.

In [ ]:
%pylab inline
import torch
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch import nn, optim
from VAE import *
from train import *
import numpy as np

## Run Gaussian model

In [ ]:
# set hyperparameters
latent_dim = 10
batch_size = 50
epochs = 50

# Load data
train_data = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor())

In [ ]:
VAE_Gaussian, loss_Gaussian, z_Gaussian, KL_Gaussian, log_bern_Gaussian = run_train(latent_dim, epochs, 'Gaussian', train_data, 1e-3)

In [ ]:
VAE_gumbel, loss_gumbel, z_gumbel, KL_gumbel, log_bern_gumbel = run_train(latent_dim, epochs, 'Gumbel', train_data, 1e-3)

In [ ]:
VAE_logit, loss_logit, z_logit, KL_logit, log_bern_logit = run_train(latent_dim, epochs, 'logit', train_data, 1e-3)

In [ ]:
import random
random.seed(1000)

VAE_logit_rank1, loss_logit_rank1, z_logit_rank1, KL_logit_rank1, log_bern_rank1 = run_train(latent_dim, epochs, 'logit', train_data, 1e-3, True)

In [ ]:
VAE_logit_sigmoid, loss_logit_sigmoid, z_logit_sigmoid, KL_logit_sigmoid, log_bern_sigmoid = run_train(latent_dim, epochs, 'logit-sigmoidal', train_data, 1e-3)

## Plot of losses

In [ ]:
epoch_space = np.linspace(1,50,50)
plt.plot(epoch_space, loss_Gaussian, label='Gaussian')
plt.plot(epoch_space, loss_logit, label='logit')
plt.plot(epoch_space, loss_logit_rank1, label='logit rank1')
plt.plot(epoch_space, loss_logit_sigmoid, label='logit sigmoidal')
plt.plot(epoch_space, loss_gumbel, label='Gumbel')
plt.legend()
plt.show()

## T-SNE plots

In [ ]:
from sklearn.manifold import TSNE 
import collections
from collections import *

def find_TSNE_coordinates(train_loader_scatter_plot, model):
    
    x_coordinates = defaultdict(lambda: [])
    y_coordinates = defaultdict(lambda: [])
    
    for batch_idx, (data, label) in enumerate(train_loader_scatter_plot):
        _, z, _ = model(data)
        labels = label.data.cpu().numpy()
        embedded = TSNE(n_components=2).fit_transform(z.data.cpu().numpy())
        if batch_idx == 0:
            break
            
    assert embedded.shape[0] == labels.shape[0]    
    for i in range(embedded.shape[0]):
        x_coordinates[labels[i]].append(embedded[i][0])
        y_coordinates[labels[i]].append(embedded[i][1])
        
    return x_coordinates, y_coordinates

In [ ]:
train_data = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=10000, shuffle=False, **{})

In [ ]:
x_latent_space_Gumbel, y_latent_space_Gumbel = find_TSNE_coordinates(train_loader, VAE_gumbel)
for label in x_latent_space_Gumbel:
    plt.scatter(x_latent_space_Gumbel[label], y_latent_space_Gumbel[label], marker='.')
plt.show()

In [ ]:
x_latent_space_Gaussian, y_latent_space_Gaussian = find_TSNE_coordinates(train_loader, VAE_Gaussian)
for label in x_latent_space_Gaussian:
    plt.scatter(x_latent_space_Gaussian[label], y_latent_space_Gaussian[label], marker='.')
plt.show()

In [ ]:
x_latent_space_logit, y_latent_space_logit = find_TSNE_coordinates(train_loader, VAE_logit)
for label in x_latent_space_logit:
    plt.scatter(x_latent_space_logit[label], y_latent_space_logit[label], marker='.')
plt.show()

In [ ]:
x_latent_space_logit, y_latent_space_logit = find_TSNE_coordinates(train_loader, VAE_logit_sigmoid)
for label in x_latent_space_logit:
    plt.scatter(x_latent_space_logit[label], y_latent_space_logit[label], marker='.')
plt.show()

In [ ]:
x_latent_space_logit, y_latent_space_logit = find_TSNE_coordinates(train_loader, VAE_logit_rank1)
for label in x_latent_space_logit:
    plt.scatter(x_latent_space_logit[label], y_latent_space_logit[label], marker='.')
plt.show()

## Histograms of z

In [ ]:
# def plot_histogram(title, z):
#     hist_values = [[] for i in range(z.size(0))]
#     for i in range(z.size(0)):
#         for j in range(z.size(1)):
#             hist_values[i].append(float(z[i][j]))
#             plt.hist(hist_values[i],  bins = 10, histtype=u'step')
#     plt.title(title)
#     plt.show()

def plot_histogram(title, z):
    hist_values = [[] for i in range(len(z))]
    
    for i in range(z[0].size(0)):
        for j in range(len(z)):
            hist_values[i].append(float(z[j][i]))
        plt.hist(hist_values[i],  bins = 10, histtype=u'step')
        plt.show()

In [ ]:

plot_histogram('Histogram for Gaussian Distribution', z_Gaussian)


In [ ]:

plot_histogram('Histogram for logit Distribution', z_logit)


In [ ]:

plot_histogram('Histogram for the Logit Distribution for Rank1 Covariance Approximation', z_logit_rank1)


In [ ]:

plot_histogram('Histogram for the Gumbel Distribution', z_gumbel)


In [ ]:

plot_histogram('Histogram for logit_sigmoid', z_logit_sigmoid)


In [ ]:
print ('KL mean for Gauss = ', torch.mean(torch.stack(KL_Gaussian)).data.cpu().numpy())
print ('KL mean for Gumbel = ', torch.mean(torch.stack(KL_gumbel)).data.cpu().numpy())
print ('KL mean for logit = ',torch.mean(torch.stack(KL_logit)).data.cpu().numpy())
print ('KL mean for logit_rank1 = ', torch.mean(torch.stack(KL_logit_rank1)).data.cpu().numpy())
print ('KL mean for logit_sigmoid = ', torch.mean(torch.stack(KL_logit_sigmoid)).data.cpu().numpy())

In [ ]:
print ('log_bern mean for Gaussian = ' ,torch.mean(torch.stack(log_bern_Gaussian)).data.cpu().numpy())
print ('log_bern mean for Gumbel = ', torch.mean(torch.stack(log_bern_gumbel)).data.cpu().numpy())
print ('log_bern mean for Logit = ', torch.mean(torch.stack(log_bern_logit)).data.cpu().numpy())
print ('log_bern mean for Logit_rank1 = ', torch.mean(torch.stack(log_bern_rank1)).data.cpu().numpy())
print ('log_bern mean for Logit_sigmoid = ', torch.mean(torch.stack(log_bern_sigmoid)).data.cpu().numpy())